<a href="https://www.kaggle.com/code/dariopaez/tps-oct22-final-submission?scriptVersionId=111965941" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Final submission for **Tabular Playground Series - Oct 2022** 

# 1. Introduction

This notebook documents my work throughout Tabular Playground Series competition. The main objective is to collect ideas, different approaches, difficulties faced and briefly explain my thought process around this challenge. It is also an invitation to review the challenge and think about what was done correctly, what kind of mistakes were made and what could have been done in a different way in order to get better results. 


### Libraries

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import os
import gc
import pandas as pd
import tensorflow as tf
import numpy as np
import seaborn as sns
sns.set(style='darkgrid', font_scale=1.6)
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow import keras
from tensorflow.keras import layers
pd.options.display.max_columns = None
from sklearn.model_selection import train_test_split
import random
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential

/kaggle/input/tabular-playground-series-oct-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-oct-2022/train_9.csv
/kaggle/input/tabular-playground-series-oct-2022/train_3.csv
/kaggle/input/tabular-playground-series-oct-2022/train_4.csv
/kaggle/input/tabular-playground-series-oct-2022/test_dtypes.csv
/kaggle/input/tabular-playground-series-oct-2022/train_8.csv
/kaggle/input/tabular-playground-series-oct-2022/train_dtypes.csv
/kaggle/input/tabular-playground-series-oct-2022/train_5.csv
/kaggle/input/tabular-playground-series-oct-2022/train_2.csv
/kaggle/input/tabular-playground-series-oct-2022/test.csv
/kaggle/input/tabular-playground-series-oct-2022/train_0.csv
/kaggle/input/tabular-playground-series-oct-2022/train_7.csv
/kaggle/input/tabular-playground-series-oct-2022/train_1.csv
/kaggle/input/tabular-playground-series-oct-2022/train_6.csv
/kaggle/input/rocket-league-submissions/Submission_18.csv
/kaggle/input/rocket-league-submissions/Submission_43.csv
/kaggle/input/

# 2. EDA

Comments and dataset first impressions are presented in this notebook: [TPS-Oct22-Dataset First Impressions](https://www.kaggle.com/code/dariopaez/tps-oct22-dataset-first-impressions).

Besides, these works have great information and insights about this challenge data:
- [🚀 TPS Oct 22 - Rocket League EDA](https://www.kaggle.com/code/samuelcortinhas/tps-oct-22-rocket-league-eda/notebook#3.-EDA)  by Samuel Cortinhas
- [TPS OCT 2022 - EDA + Simple Models](https://www.kaggle.com/code/craigmthomas/tps-oct-2022-eda-simple-models/notebook) by Craig Thomas 

These presents a great exploratory analysis. 

From EDA we can extract some general conclusions:

- Dataset needs to be compressed
- Target is unbalanced (90% for no team scoring, 5% for team A scoring and 5% for team B scoring)
- Missing values that needs to be filled are associated to players position, velocity and booster remaining


After these first impressions I tried different approaches to balance the data. These are shown [here](https://www.kaggle.com/code/dariopaez/tps-oct22-dataset-first-impressions). Then, I spent quite some time trying different models and parameters using the new balanced dataset. However, these models gave me bad results despite changing their structure and parameters. Since competition submission are evaluated by LogLoss, better results were obtained without modifying the data distribution and, probably, the test set distribution is similar to the train set. This explanation is better explained and detailed in this work: [How to kill all your efforts? ](https://www.kaggle.com/code/alexryzhkov/how-to-kill-all-your-efforts) by Alexander Ryzhkov. Thanks to his work I finally understood that I was wasting time and needed to take a different approach. 

# 3. Data Processing

Feature engineering and data compression were handled together: [TPS-Oct22-Data compression and FE](https://www.kaggle.com/code/dariopaez/tps-oct22-data-compression-and-fe). Through an iteration over the training set and saving each file into the compressed feather format. 

### Feature Engineering
Some added features included:

- Distance between Ball and both goals
- Ball speed
- Players speed
- Distance between team A players and team B goal
- Distance between team B players and team A goal
- Players to ball distance

Dropped columns:

- 'team_scoring_next'
- 'player_scoring_next'
- 'event_time'
- 'event_id'
- 'game_num'

### Missing Values 

After trying different ideas to fill missing values I ended up filling all values with 0. Other approaches did not present noticeable better results and replacing with zeros was easier. 

In brief, when a player is demolished, his position, speed and remaining booster is set to 0 as if the player is in the center of the field. 

### Useful notebooks

- Another compressing solution using Dask: [🚀How to load 21M rows in 1 minute using 2 lines](https://www.kaggle.com/code/donatoriccio/how-to-load-21m-rows-in-1-minute-using-2-lines) by Donatio Riccio

- FE and compressing with feather: [TPS-OCT-2022 - Simple TF](https://www.kaggle.com/code/eavelardev/tps-oct-2022-simple-tf/notebook) by Eduardo Avelar

# 4. Model

Model with best result achieved has the following features:

- Compressed dataset in feather format
- Added computed distances and speed features 
- Missing values replaced by 0s 
- Data transformation to make each column distribution to have mean 0 and standard deviation 1
- 4 Dense layers with Relu activation
- Added Normalization and Dropout layers
- 2 Output dense neurons with sigmoid activation

Despite **compressing** the data, I could not train a model with the whole dataset without running **out of memory**. Because of that, I made a loop to mix 5% samples from each train file and use that **mixed file** to train each model. With another loop, I generated different versions of that mixed train file and train several models. After training for one epoch I used the model to make predictions on the test set and saved results into csv files. The idea was to compute the mean of several predictions and analyze if that process gave better results than training only one model for more epochs. 

### Cells below are to pre-process the test set and iteration to obtain several predictions

In [2]:
data_path = '../input/tps-oct-22-feather'
submissions_path = '../input/rocket-league-submissions'

In [3]:
#List of functions

def demolitions(df):
    df['active_players_A'] = 3-(df['p0_pos_x'].isna()).astype(int)-(df['p1_pos_x'].isna()).astype(int)-(df['p2_pos_x'].isna()).astype(int)
    df['active_players_B'] = 3-(df['p3_pos_x'].isna()).astype(int)-(df['p4_pos_x'].isna()).astype(int)-(df['p5_pos_x'].isna()).astype(int)
    return df

#Computes distances, speeds and also calculates the maximum distance to ball in order to replace NaN values in players positions
def compute_df(df):
    # Estimates
    goalA_coord = (0,-103.5, 6)
    goalB_coord = (0,103.5, 6)
    df.fillna(0, inplace=True)
    
    # Euclidean distance. Ball to goals distances
    df['ball_dist_to_goalA'] = np.sqrt((df['ball_pos_x']-goalA_coord[0])**2 + (df['ball_pos_y']-goalA_coord[1])**2 + (df['ball_pos_z']-goalA_coord[2])**2)
    df['ball_dist_to_goalB'] = np.sqrt((df['ball_pos_x']-goalB_coord[0])**2 + (df['ball_pos_y']-goalB_coord[1])**2 + (df['ball_pos_z']-goalB_coord[2])**2)
    
    #Ball speed
    df['ball_speed'] = np.sqrt(df['ball_vel_x']**2 + df['ball_vel_y']**2 + df['ball_vel_z']**2) 
    
    #Players speeds
    for i in range(6):
        df[f'p{i}_speed'] = np.sqrt(df[f'p{i}_vel_x']**2 + df[f'p{i}_vel_y']**2 + df[f'p{i}_vel_z']**2)
        
        
    #Team A distance to Goal B
    for i in range(3):
        df[f'p{i}_dist_to_goal'] = np.sqrt((df[f'p{i}_pos_x'] - goalB_coord[0])**2 +
                                           (df[f'p{i}_pos_y'] - goalB_coord[1])**2 +
                                           (df[f'p{i}_pos_z'] - goalB_coord[2])**2)
        
        
    #Team B distance to Goal A
    for i in range(3, 6):
        df[f'p{i}_dist_to_goal'] = np.sqrt((df[f'p{i}_pos_x'] - goalA_coord[0])**2 +
                                           (df[f'p{i}_pos_y'] - goalA_coord[1])**2 +
                                           (df[f'p{i}_pos_z'] - goalA_coord[2])**2)
        
        
    #Player to Ball distance
    
    for i in range(6):
        df[f'p{i}_dist_to_ball'] = np.sqrt((df[f'p{i}_pos_x']-df['ball_pos_x'])**2 + (df[f'p{i}_pos_y']-df['ball_pos_y'])**2 + (df[f'p{i}_pos_z']-df['ball_pos_z'])**2)
        
        
    return df

In [4]:

# #Pre-Process test set
# test_df = pd.read_feather('../input/tps-oct-22-feather/test.feather')
# test_df = demolitions(test_df)
# test_dt = compute_df(test_df)
# test_df.drop(columns=['id'], inplace=True)


In [5]:
# # 10 times loop in order to obtain ten submission files from different models and different mixed data

# for i in range(10):
    
#     #Get a train mix file with a random 5% of each train file
#     train_dfs = []
#     for j in range(10):
#         print(f'############## Writing file {j} ##############')
#         train_file = os.path.join(data_path, f'train_{j}.feather')
#         train = pd.read_feather(train_file).sample(frac=0.05)

#         #Append each file
#         train_dfs.append(train)

#     pd.concat(train_dfs).reset_index(drop=True).to_feather('train_mix.feather')
#     del train_dfs
#     gc.collect()
    
#     #Load train mix
#     train_df = pd.read_feather('./train_mix.feather')
    
#     #Drop game_num
#     train_df.drop(columns=['game_num'], inplace=True)
    
#     #Define targets
#     targets_df = pd.DataFrame()
#     targets_df['team_A'] = train_df.pop('team_A_scoring_within_10sec')
#     targets_df['team_B'] = train_df.pop('team_B_scoring_within_10sec')
#     print('team_A mean: ', targets_df['team_A'].mean())
#     print('team_B mean: ', targets_df['team_B'].mean())
    
#     #Standarization
#     scaler = StandardScaler()
#     scaler_cols = [x for x in train_df.columns if x not in ['active_players_A','active_players_B']]

#     train_estandar = pd.DataFrame(scaler.fit_transform(train_df[scaler_cols]))
#     train_estandar.columns = scaler_cols
#     train_estandar.index = train_df.index

#     train_active_players = train_df.drop(columns=scaler_cols)
#     train_final = pd.concat([train_active_players, train_estandar], axis=1)
    
#     test_estandar = pd.DataFrame(scaler.transform(test_df[scaler_cols]))
#     test_estandar.columns = scaler_cols

#     test_active_players = test_df.drop(columns=scaler_cols)
#     test_final = pd.concat([test_active_players, test_estandar], axis=1)
    
#     del train_active_players, train_estandar, train_df, test_estandar, test_active_players
#     gc.collect()
    
#     #Model Training
#     model = Sequential([
#     tf.keras.layers.BatchNormalization(input_shape = [len(train_final.columns)]),
#     tf.keras.layers.Dense(128, activation='relu'),
    
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dropout(0.05),
#     tf.keras.layers.Dense(256, activation='relu'),
    
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dropout(0.1),
#     tf.keras.layers.Dense(256, activation='relu'),
 
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dropout(0.15),
#     tf.keras.layers.Dense(128, activation='relu'),    
    
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(2, activation='sigmoid')
#     ])
    
#     model.compile(optimizer=tf.keras.optimizers.Adam(),
#              loss='binary_crossentropy',
#              metrics=['accuracy'])
    
#     print(f'############## Training model number {i} ##############')
#     history = model.fit(train_final, 
#                     targets_df, 
#                     epochs=1, 
#                     batch_size=32)
    
#     #model prediction
#     model.trainable = False
#     model.compile(optimizer=tf.keras.optimizers.Adam(),
#              loss='binary_crossentropy',
#              metrics=['accuracy'])
    
#     preds = model.predict(test_final)
    
#     ss = pd.read_csv('../input/tabular-playground-series-oct-2022/sample_submission.csv')
#     ss['team_A_scoring_within_10sec'] = preds[:,0]
#     ss['team_B_scoring_within_10sec'] = preds[:,1]
#     ss.to_csv(f'Submission_{i}.csv', index=False)
    
#     print(ss['team_A_scoring_within_10sec'].mean())
#     print(ss['team_B_scoring_within_10sec'].mean())
    
#     del model, preds, test_final, train_final, targets_df
#     gc.collect()
    


# 5. Final submission

In [6]:
submission_df = pd.DataFrame()

for i in range(60):
    sub = pd.read_csv(f'../input/rocket-league-submissions/Submission_{i}.csv', encoding='utf-8')
    
    submission_df = pd.concat([submission_df, sub.iloc[:,1:]], axis=1)

#submission_df.head()

,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec,team_A_scoring_within_10sec,team_B_scoring_within_10sec
0,0.047446,0.033479,0.040184,0.036366,0.046406,0.057984,0.045069,0.031126,0.035225,0.039970,0.027195,0.048541,0.027200,0.028392,0.027673,0.048533,0.037778,0.036896,0.032942,0.041691,0.028414,0.051926,0.041556,0.037300,0.030937,0.050768,0.040493,0.036702,0.031805,0.027944,0.032196,0.042786,0.036087,0.042211,0.033522,0.029554,0.044364,0.032030,0.042755,0.044384,0.025149,0.048997,0.040662,0.024878,0.053280,0.048896,0.036251,0.040606,0.046628,0.037821,0.036433,0.028132,0.039919,0.035583,0.030232,0.051292,0.040617,0.037485,0.036187,0.037790,0.041541,0.032062,0.033492,0.027579,0.032618,0.046243,0.036377,0.037769,0.042234,0.027613,0.03200

In [7]:
ss = pd.read_csv('../input/tabular-playground-series-oct-2022/sample_submission.csv')
ss['team_A_scoring_within_10sec'] = submission_df['team_A_scoring_within_10sec'].mean(axis=1)
ss['team_B_scoring_within_10sec'] = submission_df['team_B_scoring_within_10sec'].mean(axis=1)
ss.to_csv(f'Submission.csv', index=False)

print(ss['team_A_scoring_within_10sec'].mean())
print(ss['team_B_scoring_within_10sec'].mean())

0.055624519127878524
0.05697068673398515


# 6. Final Comments

- I didn´t try **Data augmentation** solutions like:
 - Shuffle players in the same team
 - Flipping X axis (symmetrical)
 - Mirror Team A and Team B
 
 
 Some of these solutions could be applied with a custom Data Generator from Keras in this notebook: [TPS October 2022 - Keras/TF Neural Network](https://www.kaggle.com/code/fabianbong/tps-october-2022-keras-tf-neural-network) by Fabian Bong
 
- Since training data was grouped by game and event, a correct way to split into train and validation sets would be using Group K Fold. This is explained here: [➡️Understanding the data + why you need GroupKFold](https://www.kaggle.com/code/donatoriccio/understanding-the-data-why-you-need-groupkfold) by Donato Riccio.

- Calibration to correct the effect of sampling data:
[Calibration is all you need!](https://www.kaggle.com/code/alexryzhkov/calibration-is-all-you-need/notebook) by Alexander Ryzhkov
